In [26]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [27]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text  

In [28]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [29]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [30]:
vocab = pd.read_csv("../static/model/vocabulary.txt", header=None)
tokens = vocab[0].tolist()

In [31]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [32]:
def preprocessing(text):
    # Create a DataFrame with the input text
    data = pd.DataFrame([text], columns=['tweet'])
    
    # Convert uppercase to lowercase
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove Links
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    
    # Remove punctuations
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    
    # Remove Numbers
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    
    # Remove Stopwords
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    
    # Stemming
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    
    return data["tweet"]

In [33]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        # Initialize a zero vector with the same length as the vocabulary
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            # Check if the vocabulary word is in the sentence
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        # Append the vector to the list
        vectorized_list.append(sentence_list)

    # Convert the list of vectors into a NumPy array
    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)

    return vectorized_list_new


In [39]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction ==1:
        return 'negative'
    else:
        return 'positive'

In [42]:
txt = 'good product, awesome'
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'